# 特征选择

## 计算特征之间的相关系数

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import feather
import shutil
%matplotlib inline

In [2]:
data_path = "/home/liuhaozhe/SourceCode/PycharmProjects/User_churn_alert-main/data"
img_path = "/home/liuhaozhe/SourceCode/PycharmProjects/User_churn_alert-main/img"
os.chdir(data_path)
if os.path.exists(img_path):
    shutil.rmtree(img_path)
    os.mkdir(img_path)

In [3]:
X_train = feather.read_dataframe("X_train_cleaned.feather")
X_test = feather.read_dataframe("X_test_cleaned.feather")
y_train = feather.read_dataframe("y_train_cleaned.feather")
y_test = feather.read_dataframe("y_test_cleaned.feather")
X_train.shape, X_test.shape

((755155, 226), (323639, 226))

In [4]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

In [5]:
def plot_corr(corr_, fig_name):
    fig, ax = plt.subplots()
    fig.set_size_inches(30, 30)
    sns.heatmap(corr_, vmin=0, vmax=1)
    plt. savefig(os.path.join(img_path, f"{fig_name}.png"), dpi=300)

In [6]:
corr = X_train.corr().abs()
# plot_corr(corr, "corr_all")

In [7]:
corr_all = corr.copy()

将特征分为内部相关的几个特征群体，再对每个特征群体使用随机森林分类，得到一个特征群体中所有特征的重要性，并筛选丢弃重要性低的特征，并且将上述过程不断迭代

In [8]:
corr = corr.unstack()
corr = corr.sort_values(ascending=False)
corr = pd.DataFrame(corr).reset_index()
corr.columns = ['feat1', 'feat2', 'corr']

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from itertools import compress

In [10]:
dropped_feats = []  # 丢弃的特征，每个loop会不断增加
selected_feats = list(X_train.columns)  # 被选择的特征，每个loop会不断减少
thres = 0.9

In [11]:
for loop in range(100):
    corr_temp = corr.copy()
    corr_temp = corr_temp[corr_temp["corr"] < 1]
    corr_temp = corr_temp[corr_temp["corr"] >= thres]
    
    grouped_feats = []  # buffer
    corr_groups = []  # buffer

    i = 0  # 作为存储heatmap的图片名字

    for feat in corr_temp.feat1.unique():
        if feat not in grouped_feats:
            i += 1
            corr_block = corr_temp[(corr_temp.feat1 == feat)]

            # 绘制一个特征群体内，不同特征之间的相关系数heatmap
    #         plot_corr(corr_all.loc[list(corr_block.feat2.unique()) + [feat], list(corr_block.feat2.unique()) + [feat]].abs(), f"lvl1_{i}")
            grouped_feats += list(corr_block.feat2.unique()) + [feat]  # 累加已经属于某个特征相关群体的特征 

            corr_groups.append(corr_block)  # 增加该特征群体，用于后续使用随机森林分类，对该群体的特征进行筛选


    for group in corr_groups:
        feats = list(group.feat1.unique()) + list(group.feat2.unique())
    #     print(f"最初的特征：{feats}")
        rfc_sel = SelectFromModel(RandomForestClassifier(n_estimators=20, random_state=1, max_depth=4))
        rfc_sel.fit(X_train[feats], y_train.values.ravel())

        rfc_sel_support = np.array(rfc_sel.get_support())  # 得到特征筛选的bool序列
        dropped_feats += list(compress(feats, ~rfc_sel_support))  # 增加丢弃的特征
    #     print(f"筛选的特征：{list(compress(feats, rfc_sel_support))}")
    
    dropped_feats = list(set(dropped_feats))  # 去掉重复的特征
    selected_feats = [i for i in selected_feats if i not in dropped_feats]  # selected_feats中去掉丢弃的特征

    # 更新corr，过滤掉被丢弃特征的所有相关系数
    corr = corr[corr["feat1"].isin(selected_feats) & corr["feat2"].isin(selected_feats)]

    thres *= 0.99
    print(f"被选择的特征个数为{len(selected_feats)}, 被丢弃的特征个数为{len(dropped_feats)}")
    if len(selected_feats) <= 40:
        break

被选择的特征个数为175, 被丢弃的特征个数为51
被选择的特征个数为153, 被丢弃的特征个数为73
被选择的特征个数为144, 被丢弃的特征个数为82
被选择的特征个数为132, 被丢弃的特征个数为94
被选择的特征个数为121, 被丢弃的特征个数为105
被选择的特征个数为111, 被丢弃的特征个数为115
被选择的特征个数为102, 被丢弃的特征个数为124
被选择的特征个数为95, 被丢弃的特征个数为131
被选择的特征个数为92, 被丢弃的特征个数为134
被选择的特征个数为85, 被丢弃的特征个数为141
被选择的特征个数为78, 被丢弃的特征个数为148
被选择的特征个数为73, 被丢弃的特征个数为153
被选择的特征个数为72, 被丢弃的特征个数为154
被选择的特征个数为70, 被丢弃的特征个数为156
被选择的特征个数为66, 被丢弃的特征个数为160
被选择的特征个数为62, 被丢弃的特征个数为164
被选择的特征个数为61, 被丢弃的特征个数为165
被选择的特征个数为57, 被丢弃的特征个数为169
被选择的特征个数为53, 被丢弃的特征个数为173
被选择的特征个数为52, 被丢弃的特征个数为174
被选择的特征个数为50, 被丢弃的特征个数为176
被选择的特征个数为50, 被丢弃的特征个数为176
被选择的特征个数为48, 被丢弃的特征个数为178
被选择的特征个数为46, 被丢弃的特征个数为180
被选择的特征个数为44, 被丢弃的特征个数为182
被选择的特征个数为42, 被丢弃的特征个数为184
被选择的特征个数为39, 被丢弃的特征个数为187
被选择的特征个数为39, 被丢弃的特征个数为187
被选择的特征个数为39, 被丢弃的特征个数为187
被选择的特征个数为37, 被丢弃的特征个数为189
被选择的特征个数为34, 被丢弃的特征个数为192
被选择的特征个数为34, 被丢弃的特征个数为192
被选择的特征个数为33, 被丢弃的特征个数为193
被选择的特征个数为32, 被丢弃的特征个数为194
被选择的特征个数为31, 被丢弃的特征个数为195
被选择的特征个数为28, 被丢弃的特征个数为198


In [12]:
# rfc = RandomForestClassifier(n_estimators=20, random_state=1, max_depth=10)
# rfc.fit(X_train[selected_feats], y_train.values.ravel())


# importance = pd.concat([pd.Series(selected_feats), pd.Series(rfc.feature_importances_)], axis=1)
# importance.columns = ['feature', 'importance']
# importance.sort_values(by='importance', ascending=False)

In [13]:
# list(importance.sort_values(by='importance', ascending=False)["feature"])[: 30]

In [14]:
selected_feats

['life_time',
 'is_latest_version',
 'is_visitor',
 'cash_subgrade',
 'is_bought',
 'login_cnt_30d',
 'is_newbie_bundle_buyer_30d',
 'battle_pass_ads_watch_cnt',
 'chip_consume_7d',
 'chip_consume_rate_7d',
 'diamond_consume_rate_7d',
 'bandit_cnt_30d',
 'all_in_rate_3d',
 'register_country_arpu',
 'click_unpay_30d',
 'pay_time_interval_mean_180d',
 'win_per100',
 'sblind',
 'login_time_hour_list_30d_std',
 'register_time_month',
 'register_time_minute',
 'register_time_second',
 'last_login_minute',
 'last_login_second',
 'last_bankrupt_minute',
 'first_rank_time_minute',
 'login_time_minute',
 'login_time_second']